In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

/Users/joemccann/VENVs/fpENV/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
def get_csv():
    table_data = []
    
    for i in range(1, 19):
        file_path = '~/MLProjects/fantasypoints/data/adv_stats_2022/FantasyPros_Fantasy_Football_Advanced_Stats_Report_RB_{}_22.csv'.format(i)
        df = pd.read_csv(file_path)
        table_data.append(df)
    
    report_df = pd.concat(table_data, ignore_index=True)


In [ ]:
report_df.head()

In [15]:
# def get_data(url):
#     response = requests.get(url)

#     if response.status_code == 200:
#         return response.text
#     else:
#         print("Failed to retrieve the page.")

def create_advanced_stats_report(pos, week, year):
    adv_column_names = ['rank', 'player', 'games', 'att', 'yds', 
                        'y/att', 'ybcon', 'ybon/att', 'rush_yacon', 
                        'yacon/att', 'brktkl', 'tk_loss', 'tk_loss_yds', 'lng_td', 
                        '10_plus_yds', '20_plus_yds', '30_plus_yds', '40_plus_yds', 
                        '50_plus_yds', 'lng', 'rec', 'tgt', 'rz_tgt', 'rec_yacon']
    
    file_path = '~/MLProjects/fantasypoints/data/adv_stats_20{}/FantasyPros_Fantasy_Football_Advanced_Stats_Report_{}_{}_{}.csv'.format(year,pos,week,year)
    
    report_df = pd.read_csv(file_path)
    report_df.columns = adv_column_names
    report_df.columns = [column.lower() for column in report_df.columns]
    report_df['week'] = week
    report_df['year'] = year
   
    return report_df

def create_basic_stats_report(pos, week, year):
    fts_columns = ['rank', 'player','att', 'yds_rush', 'yds_rush/a', 'lg', '20_plus', 'td_rush', 'rec', 'tgt', 'yds_rec', 'yds/rec', 'td_rec', 'rl', 'g', 'fpts', 'fpts/g', 'rost']
     
    file_path = '~/MLProjects/fantasypoints/data/basic_stats_20{}/FantasyPros_Fantasy_Football_Statistics_{}_{}_{}.csv'.format(year,pos,week,year)
    
    report_df = pd.read_csv(file_path)
    report_df.columns = fts_columns
    report_df.columns = [column.lower() for column in report_df.columns]
    report_df['week'] = week
    report_df['year'] = year
   
    return report_df

def build_data_set(pos, week, year):
    adv_stats = create_advanced_stats_report(pos,week,year)
    fts_pts = create_basic_stats_report(pos,week,year)
    
    return pd.merge(adv_stats, fts_pts[['rank','fpts', 'fpts/g']], left_on='rank', right_on='rank') 

In [16]:
build_data_set('RB', 2, 22)

,rank,player,games,att,yds,y/att,ybcon,ybon/att,rush_yacon,yacon/att,...,50_plus_yds,lng,rec,tgt,rz_tgt,rec_yacon,week,year,fpts,fpts/g
0,1.0,Nick Chubb (CLE),1.0,17.0,87.0,5.1,46.0,2.7,41.0,2.4,...,0.0,22.0,3.0,3.0,0.0,0.0,2,22,29.3,29.3
1,2.0,Aaron Jones (GB),1.0,15.0,132.0,8.8,49.0,3.3,83.0,5.5,...,0.0,36.0,3.0,3.0,1.0,3.0,2,22,29.0,29.0
2,3.0,Tony Pollard (DAL),1.0,9.0,43.0,4.8,26.0,2.9,17.0,1.9,...,0.0,17.0,4.0,7.0,0.0,12.0,2,22,15.8,15.8
3,4.0,D'Andre Swift (PHI),1.0,5.0,56.0,11.2,51.0,10.2,5.0,1.0,...,1.0,50.0,2.0,5.0,0.0,2.0,2,22,14.7,14.7
4,5.0,Damien Harris (BUF),1.0,15.0,71.0,4.7,49.0,3.3,22.0,1.5,...,0.0,16.0,2.0,2.0,1.0,5.0,2,22,14.7,14.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,106.0,Julius Chestnut (TEN),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,22,-2.0,-2.0
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,22,NaN,NaN
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,22,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,22,NaN,NaN


In [17]:
combined_df = pd.DataFrame()

for i in range(22, 2024):
    for j in range(1, 19):
        df = build_data_set('RB',j,i)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/joemccann/MLProjects/fantasypoints/data/basic_stats_2023/FantasyPros_Fantasy_Football_Statistics_RB_1_23.csv'

In [ ]:
combined_df

In [ ]:
columns_to_int = ['games', 'att', 'yds','ybcon',  'rush_yacon', 
                        'brktkl', 'tk_loss', 'tk_loss_yds', 'lng_td', 
                        '10_plus_yds', '20_plus_yds', '30_plus_yds', '40_plus_yds', 
                        '50_plus_yds', 'lng', 'rec', 'tgt', 'rz_tgt', 'rec_yacon', 'week','year','fpts', 'fpts/g']

columns_to_float = ['ybon/att','ybon/att','yacon/att', 'brktkl', 'tk_loss', 'tk_loss_yds', 'lng_td', 
                        '10_plus_yds', '20_plus_yds', '30_plus_yds', '40_plus_yds', 
                        '50_plus_yds', 'lng', 'rec', 'tgt', 'rz_tgt', 'rec_yacon', 'week','year','fpts', 'fpts/g']

combined_df[columns_to_change] = combined_df[columns_to_change].astype('int64')
combined_df[columns_to_change] = combined_df[columns_to_change].astype('float64')


In [ ]:
player_data = combined_df[(combined_df['player'] == 'Josh Jacobs (LV)') & (combined_df['year'] == 2020)]

In [ ]:
player_data.dtypes